In [1]:
from sklearn.datasets import load_files
from sklearn.model_selection import train_test_split
import numpy as np
from glob import glob
from PIL import ImageFile
import random
from tqdm import tqdm
from keras.preprocessing import image
from keras.utils import np_utils
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Dropout, Flatten, Dense
from keras.models import Sequential
from keras.callbacks import ModelCheckpoint

Using TensorFlow backend.


In [2]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [3]:
import keras
!pip install h5py
!pip install simplejson

In [4]:
import math
import keras
import numpy as np
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Flatten, Input, AveragePooling2D, merge, Activation,GlobalAveragePooling2D
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization,SeparableConv2D
from keras.layers import Concatenate
from keras.regularizers import l1 ,l2
from keras.callbacks import ReduceLROnPlateau,ModelCheckpoint
from keras.optimizers import SGD,Adam

In [5]:
num_classes=43
#(x_train, y_train), (x_test, y_test) = cifar10.load_data()
img_height, img_width, channel = 200,200,3

In [6]:
# Hyperparameters
batch_size = 32 # number of batches per epoch 
num_classes = 43 #number of classifictions 
epochs = 50 #number of times each image is scanned 
l = 10 # growth rate of network
num_filter = 12 # number of filters 
compression = 0.9
#dropout_rate = 0.0

In [7]:
# Dense Block
def add_denseblock(input, num_filter, dropout_rate):
    global compression
    temp = input
    for _ in range(l):
        BatchNorm = BatchNormalization()(temp)
        relu = Activation('relu')(BatchNorm)
        Conv2D_3_3 = Conv2D(int(num_filter*compression), (3,3), use_bias=False ,padding='same')(relu)
        if dropout_rate>0:
          Conv2D_3_3 = Dropout(dropout_rate)(Conv2D_3_3)
        concat = Concatenate(axis=-1)([temp,Conv2D_3_3])
        
        temp = concat
        
    return temp

In [8]:
def add_transition(input, num_filter, dropout_rate):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    Conv2D_BottleNeck = Conv2D(int(num_filter*compression), (1,1), use_bias=False ,padding='same')(relu)
    if dropout_rate>0:
        Conv2D_BottleNeck = Dropout(dropout_rate)(Conv2D_BottleNeck)
    avg = AveragePooling2D(pool_size=(2,2))(Conv2D_BottleNeck)
    
    return avg

In [9]:
def output_layer(input):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    AvgPooling = AveragePooling2D(pool_size=(2,2))(relu)
    flat = Flatten()(AvgPooling)
    output = Dense(num_classes, activation='sigmoid',name='predictions')(flat)
    
    return output

In [10]:
num_filter = 12
dropout_rate = 0.25 #as images are small its critical to learn all features.
l = 6
input = Input(shape=(img_height, img_width, channel,))
First_Conv2D = Conv2D(num_filter, (3,3), use_bias=False ,padding='same')(input)

First_Block = add_denseblock(First_Conv2D, num_filter, dropout_rate)
First_Transition = add_transition(First_Block, num_filter, dropout_rate)

Second_Block = add_denseblock(First_Transition, num_filter, dropout_rate)
Second_Transition = add_transition(Second_Block, num_filter, dropout_rate)

#Third_Block = add_denseblock(Second_Transition, num_filter, dropout_rate)
#Third_Transition = add_transition(Third_Block, num_filter, dropout_rate)


Last_Block = add_denseblock(Second_Transition,  num_filter, dropout_rate)
output = output_layer(Last_Block)

In [11]:
model = Model(inputs=[input], outputs=[output])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 200, 200, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 200, 200, 12) 324         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 200, 200, 12) 48          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 200, 200, 12) 0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (

In [12]:
from keras.callbacks import LearningRateScheduler
# learning rate are updated as per number of epocs
def lr_schedule(epoch):
    lrate = 0.001
    if epoch > 75:
        lrate = 0.01
    elif epoch > 100:
        lrate = 0.0001       
    return lrate
 
sgd = keras.optimizers.SGD(lr=0.1)
model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

In [13]:
from keras.callbacks import ModelCheckpoint

# To save model weights for best accuracy model used callbacks and check points 
filepath='/data'

checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint,LearningRateScheduler(lr_schedule)]

In [14]:
# determine Loss function and Optimizer
model.compile(loss='categorical_crossentropy',
            optimizer=sgd,
              metrics=['accuracy'])

In [15]:
filepath="data/weights_densenetbc_2.best.hdf5"

In [16]:
from keras.preprocessing import image
#data augmentation
datagen_train = image.ImageDataGenerator(rescale = 1./255)
datagen_valid = image.ImageDataGenerator(rescale = 1./255)

In [17]:
def my_gen(gen):
    while True:
        try:
            data, labels = next(gen)
            yield data, labels
        except:
            pass

In [18]:
learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', 
                                            patience=3, 
                                            verbose=1, 
                                            factor=0.1, 
                                            min_lr=0.0001)
checkpoint_save = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')

callbacks_list = [checkpoint_save,learning_rate_reduction]

In [19]:
model.compile(loss='categorical_crossentropy',
            optimizer=sgd,
              metrics=['accuracy'])

In [20]:
val_gen = datagen_valid.flow_from_directory(directory="data/valid",target_size=(200, 200),color_mode="rgb",batch_size=32,class_mode="categorical",shuffle=True)

train_gen = datagen_train.flow_from_directory(directory="training-data/train",target_size=(200, 200),
    color_mode="rgb",batch_size=32,class_mode="categorical",shuffle=True)

Found 14506 images belonging to 43 classes.
Found 458346 images belonging to 43 classes.


In [21]:
# this part will prevent tensorflow to allocate all the avaliable GPU Memory
# backend

# Don't pre-allocate memory; allocate as-needed
import tensorflow as tf
from keras import backend as k
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

# Create a session with the above options specified.
k.tensorflow_backend.set_session(tf.Session(config=config))

In [ ]:
model.load_weights(filepath)
keras.backend.get_session().run(tf.initialize_all_variables())
model.fit_generator(my_gen(train_gen),
                    steps_per_epoch=1000,
                    epochs=15,
                    validation_data=my_gen(val_gen),
                    validation_steps=250,
                    callbacks=callbacks_list) 

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Epoch 1/15
1000/1000 [==============================] - 173s 173ms/step - loss: 3.3106 - acc: 0.1441 - val_loss: 3.2069 - val_acc: 0.1579

Epoch 00001: val_acc improved from -inf to 0.15787, saving model to data/weights_densenetbc_2.best.hdf5
Epoch 2/15
1000/1000 [==============================] - 167s 167ms/step - loss: 3.0237 - acc: 0.2057 - val_loss: 2.8523 - val_acc: 0.2398

Epoch 00002: val_acc improved from 0.15787 to 0.23978, saving model to data/weights_densenetbc_2.best.hdf5
Epoch 3/15
1000/1000 [==============================] - 167s 167ms/step - loss: 2.8137 - acc: 0.2296 - val_loss: 2.8326 - val_acc: 0.2335

Epoch 00003: val_acc did not improve from 0.23978
Epoch 4/15
1000/1000 [==============================] - 167s 167ms/step - loss: 2.7481 - acc: 0.2324 - val_loss: 2.9317 - val_acc: 0.2367

Epoch 00004: val_acc did not improve from 0.23978
Epoch 5/15
1000/1000 [==============================] - 167

In [ ]:
model.fit_generator(my_gen(train_gen),
                    steps_per_epoch=14323,
                    epochs=10,
                    validation_data=my_gen(val_gen),
                    validation_steps=453,
                    callbacks=callbacks_list) 

In [ ]:
datagen = image.ImageDataGenerator(
    zoom_range=0.25, rotation_range=15.,channel_shift_range=25., width_shift_range=0.02, height_shift_range=0.02, horizontal_flip=True, fill_mode='constant'
        )

In [ ]:
model.fit_generator(datagen.flow_from_directory(directory="training-data/train",target_size=(224, 224),
    color_mode="rgb",batch_size=32,class_mode="categorical",shuffle=True),steps_per_epoch=4616,epochs=150,callbacks=callbacks_list)   
